In [29]:
#        ,----,                                          
#      ,/   .`|                                          
#    ,`   .'  :                      ,---,               
#  ;    ;     /                    .'  .' `\             
#.'___,/    ,'  ,---.      ,---,.,---.'     \    ,---.   
#|    :     |  '   ,'\   ,'  .' ||   |  .`\  |  '   ,'\  
#;    |.';  ; /   /   |,---.'   ,:   : |  '  | /   /   | 
#`----'  |  |.   ; ,. :|   |    ||   ' '  ;  :.   ; ,. : 
#    '   :  ;'   | |: ::   :  .' '   | ;  .  |'   | |: : 
#    |   |  ''   | .; ::   |.'   |   | :  |  ''   | .; : 
#    '   :  ||   :    |`---'     '   : | /  ; |   :    | 
#    ;   |.'  \   \  /           |   | '` ,/   \   \  /  
#    '---'     `----'            ;   :  .'      `----'   
#                                |   ,.'                 
#                                '---'                   
#                                                        

# COMO ME LLEGA EL CORPUS? El corpus lo leo ya sea de un txt o de un csv
# COMO Y POR Q SEPARARLO: Las mananeras vienen secuencialmente y hay periodos donde se repte, se puede separar por fechas y randomizar las oraciones dentro.
# QUE ES UNA SEMILLA DE REPRODUCIBILIDAD: 
# Hacer un corpus de prueba para las siguientes ...
# 1) Separar el corpus  el conjuntos entrenamiento y prueba . 
# Recordar que el conjunto que prueba debe ser creado de manera aleatoria, por lo tanto, 
# debe de fijarse una semilla para lograr reproducibilidad.

# COMO ANALIZAR UN CORPUS CON GRAFICAS BONITAS ...
# 2) Hacer un análisis exploratorio del conjunto de entrenamiento para decidir sobre el siguiente paso. 
# Una guía es la siguiente: https://neptune.ai/blog/exploratory-data-analysis-natural-language-processing-tools

# ---------------------------------------
# QUE DECISIONES VOY A TOMAR EN MI CORPUS
# 3) En función del análisis previo: limpiar, normalizar, segmentar y tokenizar el corpus de entrenamiento.  
# 4) Las palabras poco frecuentes transformarlas en el token <UNK>. 

# CORPUS LISTO PARA EL MODELO DE N-GRAMAS
# CUAL ES MI OUTPUT, QUE NECESITA?

# 5) Una vez armado el corpus de entrenamiento, construir el corpus ordenado, listo para ser usado en el modelo de n-gramas,
#    debe tener la estructura [['token','token', ...'token'],['token','token', ...'token'] .. ['token','token', ...'token']]. 

# PUEDO AGREGAR OTRAS TECNICAS AL VOCAB?
# 6) Crea el vocabulario

# **Obteniendo el corpus** 📖

In [30]:
def getData(filename: str) -> str:
    
    rawData = ""
    with open(filename, encoding="utf-8") as f:
        for line in f:
            rawData += line
    return rawData

In [31]:
rawCorpus = getData("corpus.txt")
print(f"Carácteres: {len(rawCorpus)}")
print(f"Extracto: \n{rawCorpus[:500]}")

Carácteres: 1218
Extracto: 
" … de su partido que había que destazar o destripar o despanzurrar, ¿o cómo era?, cualquiera de esas palabras que, finalmente, dichas o no dichas de manera directa los legisladores ilustran, describen, finalmente lo que según los especialistas y la gente que directamente está operando al sistema electoral en México pues podría ocurrir en caso de que estas reformas finalmente puedan validarse de manera sorpresiva, diría así desde mi punto de vista, por la Suprema Corte de Justicia que ya están a


## **Análisis exploratorio** 🔎

El propósito es realizar un análisis exploratorio sobre los datos crudos para tener una base en la toma de desiciones con respecto al pre-procesamiento de datos (limpieza, normalización, etc.).

### Estadísticas simples

- Cantidad de palabras (secuencias de caractéres separados por espacios).
- Cantidad de oraciones (secuencias de palabras separados por un punto).

In [32]:
from nltk.tokenize import word_tokenize, sent_tokenize
words=word_tokenize(rawCorpus, "spanish")
sentences=sent_tokenize(rawCorpus, "spanish")

print("Cantidad de carácteres:", len(rawCorpus))
print("Cantidad de palabras:", len(words))
print("Cantidad de oraciones:", len(sentences))
print("Promedio de carácteres por palabra:", round(len(rawCorpus) / len(words)))
print("Promedio de palabras por oración:", round(len(words) / len(sentences)))

Cantidad de carácteres: 1218
Cantidad de palabras: 219
Cantidad de oraciones: 6
Promedio de carácteres por palabra: 6
Promedio de palabras por oración: 36


### Distribución del tamaño de las palabras

In [33]:
import plotly.express as pt

fig = pt.violin([len(word) for word in words], box=True, title="Distribución del tamaño de las palabras")
fig.update_layout(yaxis_title="Tamaño de palabra")
fig.update_layout(xaxis_title="")
fig.show()

Quitamos los signos de punctuación para observar como cambia la distribución.

In [34]:
import plotly.express as pt
import re

def stripWord(word: str) -> str:
    """ Elimina los signos de punctuación de una palabra. """
    return re.sub('\W+','', word)

fig = pt.violin([len(word) for word in map(stripWord, words) if word], box=True, 
                title="Distribución del tamaño de las palabras sin sígnos de punctuación.")
fig.update_layout(yaxis_title="Tamaño de palabra")
fig.update_layout(xaxis_title="")
fig.show()

### Distribución del tamaño de las oraciones

In [35]:
fig = pt.violin([len(word_tokenize(sentence, "spanish")) for sentence in sentences], box=True, title="Distribución del tamaño de las oraciones")
fig.update_layout(yaxis_title="Tamaño de oración")
fig.update_layout(xaxis_title="")
fig.show()

## **Limpieza de datos** 🧹

In [36]:
def cleanWord(word: str) -> str:
    """ Elimina los signos de punctuación de una palabra excepto el punto y coma. """
    return re.sub(r'[^\w\,\.]','', word)

def cleanSentence(sentence: str) -> list:
    """ Regresa un listado de las palabras límpias de una oración. """
    words = word_tokenize(sentence)
    cleanWords = [word for word in map(cleanWord, words) if word]
    return cleanWords

In [37]:
cleanSentences = [cleanSentence(sentence) for sentence in sent_tokenize(rawCorpus, "spanish")]
nCleanWords = sum(len(word) for word in cleanSentences)
print(f"Se limpiaron {len(words) - nCleanWords} palabras.")
print("Promedio de palabras por oración aplicando limpieza:", round(  cleanWords/ len(cleanSentences)))

Se limpiaron 11 palabras.


NameError: name 'cleanWords' is not defined

### Análizando las palabras en el corpus

In [38]:
from collections import defaultdict, Counter
dic=defaultdict(int)

for sentence in cleanSentences:
    for word in sentence:
        dic[word] += 1
        
wordFreq = Counter(dic)
mostFreq = wordFreq.most_common()

In [39]:
import pandas as pd
df = pd.DataFrame(mostFreq[:10])
df.columns = ["palabra", "frecuencia"]
fig = pt.bar(df, x="palabra", y="frecuencia", color="palabra", title="Frecuencia de palabras")
fig.show()

### Sin contar las 'stopwords'

In [40]:
from nltk.corpus import stopwords
stop=set(stopwords.words('spanish'))
stop.add(".")
stop.add(",")

# Filtering stop words from dic
newDic = dict(filter(lambda pair: pair[0] not in stop, dic.items()))
wordFreq = Counter(newDic)
mostFreq = wordFreq.most_common()

df = pd.DataFrame(mostFreq[:10])
df.columns = ["palabra", "frecuencia"]
fig = pt.bar(df, x="palabra", y="frecuencia", color="palabra", title="Frecuencia de palabras sin contar stop-words")
fig.show()

### Wordcloud de palábras

In [ ]:
#%pip install wordcloud

In [49]:
from wordcloud import WordCloud, STOPWORDS

cleanText = " ".join(word for sentence in cleanSentences for word in sentence if word not in stop)

word_cloud = WordCloud(
    width=3000,
    height=2000,
    random_state=1,
    background_color="white",
    colormap="jet_r",
    collocations=False,
    stopwords=STOPWORDS,
    max_words=30
    ).generate(cleanText)

In [51]:
fig = pt.imshow(word_cloud)
fig.update_layout(coloraxis_showscale=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

## Filtrando palabras con poca ocurrencia

In [65]:
fig = pt.violin(dic.values(), box=True, title="Distribución de ocurrencia de palabras")
fig.update_layout(yaxis_title="Veces que aparece")
fig.update_layout(xaxis_title="")
fig.show()

## **Conjunto de entrenamiento y conjunto de prueba**

## Corpus final y vocabulario?